In [2]:
import os
print("Total number of dog images (Training)   : ",len(os.listdir('training\dog')))
print("Total number of cat images (Training)   : ",len(os.listdir('training\cat')))
print("Total number of dog images (Validation) : ",len(os.listdir('valid\dog')))
print("Total number of cat images (validation) : ",len(os.listdir('valid\cat')))


Total number of dog images (Training)   :  4500
Total number of cat images (Training)   :  4501
Total number of dog images (Validation) :  1000
Total number of cat images (validation) :  1001


In [3]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [5]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [6]:
from keras_preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        'training', 
        target_size=(150, 150), 
        batch_size=90,
        class_mode='binary')  

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'valid',
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 9000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:

model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=100,
)

Epoch 1/10
100/100 [==============================] - 421s 4s/step - loss: 2.2967 - accuracy: 0.5801 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 2/10
100/100 [==============================] - 142s 1s/step - loss: 0.6536 - accuracy: 0.6215 - val_loss: 0.6730 - val_accuracy: 0.6330
Epoch 3/10
100/100 [==============================] - 137s 1s/step - loss: 0.6240 - accuracy: 0.6565 - val_loss: 0.6617 - val_accuracy: 0.6810
Epoch 4/10
100/100 [==============================] - 135s 1s/step - loss: 0.5892 - accuracy: 0.6849 - val_loss: 0.5835 - val_accuracy: 0.7330
Epoch 5/10
100/100 [==============================] - 137s 1s/step - loss: 0.5713 - accuracy: 0.7027 - val_loss: 0.6730 - val_accuracy: 0.5430
Epoch 6/10
100/100 [==============================] - 136s 1s/step - loss: 0.5693 - accuracy: 0.7095 - val_loss: 0.5285 - val_accuracy: 0.7435
Epoch 7/10
100/100 [==============================] - 136s 1s/step - loss: 0.5364 - accuracy: 0.7291 - val_loss: 0.5106 - val_accuracy: 0.7420

In [9]:
tf.config.list_physical_devices('GPU') 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [15]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU
